In [1]:
import os
import time
import copy
import plotly
import numpy as np
import pandas as pd

import plotly.express as px

from enum import Enum
from typing import List, Tuple
from plotly import graph_objects as go


class WorldObject:

    draw_mode = "markers"

    def __init__(self, obj_id: str, initx: int, inity: int):
        self.obj_id = obj_id
        self.posx = initx
        self.posy = inity

    def move(self, step_x: int, step_y: int):
        self.posx += step_x
        self.posy += step_y

    def update_state(self):
        raise NotImplementedError()


class Bubble(WorldObject):

    def __init__(self, obj_id: str, initx: int, inity: int):
        super().__init__(obj_id, initx, inity)

    def update_state(self):
        direction = np.random.choice(["left", "right", "up", "down", "stay"])
        if direction == "left":
            self.move(-1, 0)
        elif direction == "right":
            self.move(1, 0)
        elif direction == "up":
            self.move(0, 1)
        elif direction == "down":
            self.move(0, -1)

    def draw(self) -> go.Scatter:

        return go.Scatter(
            x=[self.posx], y=[self.posy], mode=self.draw_mode, marker={"color": "green"}
        )

    def draw_collision(self) -> go.Scatter:
        return go.Scatter(
            x=[self.posx],
            y=[self.posy],
            mode=self.draw_mode,
            marker={"color": "red"},
        )


class Rock(WorldObject):
    def move(self):
        pass

    def update_state(self):
        pass

    def draw(self) -> go.Scatter:
        return go.Scatter(
            x=[self.posx],
            y=[self.posy],
            mode=self.draw_mode,
            marker={"color": "yellow"},
        )


class WorldGrid:
    def __init__(self, width: int, height: int, w_objects: List[WorldObject]):
        self.width = width
        self.height = height
        self.w_objects = w_objects

    def update_state(self):
        for w_object in self.w_objects:
            # only updating the state for bubbles that haven't yet collided
            if hasattr(w_object, "collided") or isinstance(w_object, Rock):
                continue
            w_object.update_state()

    def check_for_collisions(self):
        collision_coordinates = []
        
        # rocks should not collide with bubbles
        # filter bubbles coordinates
        bubble_coordinates: List[dict] = [
            {obj.posx: obj.posy} for obj in self.w_objects if not isinstance(obj, Rock)
        ]

        # checking which bubbles have collided in the new state
        for bubble in bubble_coordinates:
            for x, y in bubble.items():
                if (
                    bubble_coordinates.count({x: y}) > 1
                    and {x: y} not in collision_coordinates
                ):
                    collision_coordinates.append({x: y})
        
        # mark collisions 
        for w_object in self.w_objects:
            if {w_object.posx: w_object.posy} in collision_coordinates:
                w_object.collided = True

    def get_frame(self, frame_number: int) -> go.Frame:
        draw_objects = []

        for w_object in self.w_objects:
            if isinstance(w_object, Rock):
                continue
            elif hasattr(w_object, "collided"):
                draw_objects.append(w_object.draw_collision())
            else:
                draw_objects.append(w_object.draw())

        return go.Frame(
            data=draw_objects,
            layout=go.Layout(title_text=f"Simulation frame {frame_number}"),
        )

    def create_simulation(self, steps: int):
        frames = []

        for i in range(steps):
            self.update_state()
            self.check_for_collisions()
            frames.append(self.get_frame(i))

        fig = go.Figure(
            data=[w_obj.draw() for w_obj in self.w_objects],
            layout=go.Layout(
                xaxis=dict(range=[0, self.width], autorange=False),
                yaxis=dict(range=[0, self.height], autorange=False),
                title="Simulation",
                updatemenus=[
                    dict(
                        type="buttons",
                        buttons=[dict(label="Play", method="animate", args=[None])],
                    )
                ],
            ),
            frames=frames,
        )
        fig.show()


def simulate():
    bubbles: List[Bubble] = [
        Bubble(f"bubble_{i}_{j}", i, j)
        for i in range(5, 15, 2)
        for j in range(5, 15, 2)
    ]

    rocks: List[Rock] = [
        Rock("rock_2_2", 2, 2),
        Rock("rock_18_18", 18, 18),
        Rock("rock_18_18", 18, 18),
        Rock("rock_3_15", 3, 15),
        Rock("rock_3_14", 3, 14),
        Rock("rock_3_13", 3, 13),
        Rock("rock_5_5", 5, 5),
    ]

    world = WorldGrid(20, 20, bubbles + rocks)
    world.create_simulation(steps=50)


simulate()


/Users/luislopez/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Task: add "pops" to the code
* When bubbles collide, a "pop" should occur - one colour change should indicate the "pop" and on the next frame the bubble should be stationary and a different colour.
* Rocks should not "pop"
* The test is to see how you refactor the code to handle a new requirement.
* If there's anything you don't like about the current code - feel free to make any improvements you see fit!